<a href="https://colab.research.google.com/github/PKvasnick/PeakFit/blob/master/PeakGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generátor spekter

Tento skript generuje repliky měťených spekter k úloze A3.

Repliky spekter se generují takto: 

1. Načteme vzorky a zadáme počet replik
2. Vygenerujeme repliky spekter a uložíme je do souborového systému Colab notebooku. Repliky se generují takto:
  - Vygenerujeme náhodný čas měření v intervalu (původní čas - 10 s, původní čas + 50 s). 
  - Vyhladíme spektrum vyhlazujícím splajnem.
  - Pro každý kanál původního spektra vygenerujeme Poissonovskou intenzitu pro nové spektrum tak, že hodnotu vyhlazeného spektra přenormujeme na nový čas měření, vygenerovaný v kroku 1. 
  - Z Poissonovské intenzity vygenerujeme náhodný vzorek, a to bude hodnota generovaného spektra v tomto kanálu. 
$$
I = S_{smooth} \frac{t_{new}}{t_{orig}}, \quad
S_{new} \sim P(N|I) = \frac{I^N}{N!}e^{-I}
$$
3. Stáhneme vygenerovaná data.
4. Skript zobrazí panel plotů s původními a replikovanými spektry.

## 1. Načtení vybraných spekter
V tomto kroku načtete ze soubovorého systému vašeho počítače datové soubory pro požadované vzorky. Referenční vzorek se replikuje stejně jako ostatní vzorky a tedy nemá výsadné psotavení. Časy měření u replikovaných vzorků budou generovány náhodně. Názvy souborů očekáváme v tvaru `Vzorek-X_YYYs, kde X je identifik8tor vzorku a YYY čas nabírání dat.  

Klikněte na šípku v levém horném rohu u následující buňky pro výběr požadovaných datových souborů a když ikonka ožije, klikněte na `Prohledávat` a vyberte požadované soubory pro načtení, všechny najednou.

Po načtení se soubory zpracují - vytvoří se vyhlazená spektra s informacemi o identifikaci vzorku a času nabírání dat.

In [ ]:
#@title <-- Načtěte z počítače požadované datové soubory

from google.colab import files
import numpy as np
import pandas as pd

uploaded = files.upload() # Widget, vrací dict (název, obsah)
filenames = list(uploaded.keys())

print('Připravuji data ke generování...')
# Prepare data
import sys
from io import StringIO
from scipy.interpolate import UnivariateSpline
from numpy.random import randint

base_data = [] # Will be tuples (sample_id, time, spectrum, smooth_spectrum)

for filename, file_bytes in uploaded.items():
  seconds = int(filename[(filename.find('_')+1):filename.find('s.txt')])
  sample_id = filename[filename.find('-')+1:filename.find('_')]

  content = file_bytes.decode('utf-8')
  df = pd.read_csv(StringIO(content), sep = ' ')

  # Smoothing
  firstpeak = 6   # Ostrý pík na začátku nejde vyhladit, vyhlazujeme od něj doprava
  smoother = UnivariateSpline(df.Energy_keV[firstpeak:], df.Counts[firstpeak:], w=1.0/(1 + np.sqrt(df.Counts[firstpeak:])))
  smoothed_spectrum = np.zeros_like(df.Energy_keV)
  smoothed_spectrum[:firstpeak] = df.Counts[:firstpeak]
  smoothed_spectrum[firstpeak:] = smoother(df.Energy_keV[firstpeak:])
  np.clip(smoothed_spectrum, 0, 1000000, out = smoothed_spectrum)
  smoothed_df = pd.DataFrame({
      'Energy_keV': df.Energy_keV,
      'Smooth': smoothed_spectrum,
      'Counts': df.Counts
  })

  # Remember me
  base_data.append((sample_id, seconds, smoothed_df))

print('Hotovo.')

In [16]:
#@title <-- Zvolte požadovaný počet replik

import ipywidgets as widgets

rep_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    step=1,
    description='Replicas:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

rep_slider


IntSlider(value=1, continuous_update=False, description='Replicas:', max=20, min=1)

## 2. Generování replik

Replikovaná spektra vyrábíme takto: 
1. Náhodně vygenerujeme čas měření v intervalu (aktuální čas - 10s, aktuální čas + 50s)
2. Příslušným faktorem vynásobíme vyhlazené spektrum.
3. Replikované spektrum je Poissonnovská náhodná proměnná s intenzitou danou body škálovaného vyhlazeného spektra. 

Časy měření se pro každou repliku generují zvlášť. Tedy jednotlivé repliky téhož spektra se liší škálou (časem měření) a šumem.

### Ukládání dat
Data se ukládají do adresářů `Replicas/replica0`, `Replicas/replica1` atd. Každý takový adresář obsahuje kompletní sadu spekter.

Před naplněním generovanými daty se požadovaný adresář vytvoří, pokud neexistuje, a pokud existuje, vymažou se v něm všetchny soubory `*.txt`.

In [ ]:
#@title <-- Vygenerujte repliky

import os

nreplicas = rep_slider.value

print('Generuji {0} replik...'.format(nreplicas))

#----------------------------------------------------------------
# Připravíme grafy
#----------------------------------------------------------------
from plotly.subplots import make_subplots
from plotly import graph_objects as go

# Vypočteme počet subplotů
sample_ids = [s for s, _, _ in base_data]
titles = ['Vzorek {}'.format(s) for s in sample_ids]
ncols = 2
nsamples = len(sample_ids)
nrows = nsamples // ncols + (1 if nsamples % ncols > 0 else 0)
sample_cols = [i % ncols + 1 for i in range(nsamples)]
sample_rows = [i // ncols + 1 for i in range(nsamples)]

# Create figure

fig = make_subplots(
    rows = nrows,
    cols = ncols,
    shared_xaxes = True,
    shared_yaxes = True,
    subplot_titles = titles
    )

sample_no = 0
for sample_id, time, df in base_data:
  c = sample_cols[sample_no]
  r = sample_rows[sample_no]
  fig.add_trace(go.Scatter(
    x=df.Energy_keV, 
    y=df.Counts, 
    mode = 'lines',
    line = go.scatter.Line(color = 'red', width = 0.5),
    name = 'Counts {}'.format(sample_id)),
    row=r, col=c
    )
  if r == nrows:
    fig.update_xaxes(title_text = "Energy [keV]", row = r, col = c)
  if c == 1:
    fig.update_yaxes(title_text = "Counts", row = r, col = c)
  sample_no +=1

#----------------------------------------------------------------
# Připravíme adresáře pro repliky
#----------------------------------------------------------------

# Replicas folder
if not os.path.isdir('./Replicas'): 
  os.mkdir('./Replicas')

for replica in range(nreplicas):

  # Prepare folders
  path=''
  replica_dir = './Replicas/replica{}'.format(replica)
  if os.path.isdir(replica_dir): # clear files if dir exists
    try: 
      os.remove(path + '/*.txt')
    except OSError as error: 
      pass
  else:                           # otherwise create dir
    os.mkdir(replica_dir) 

#----------------------------------------------------------------
# Generujeme repliky
#----------------------------------------------------------------
  sample_no = 0
  for sample_id, time, df in base_data:
    # Vygenerujeme čas měření
    new_time = np.random.randint(time - 10, time + 50)
    factor = 1.0 * new_time / time
    replica_spectrum = np.random.poisson(df.Smooth * factor, len(df))
    export_df = pd.DataFrame({
      'Energy_keV': df.Energy_keV,
      'Counts': replica_spectrum
    })
    export_path = '{0}/Vzorek-{1}_{2}s.txt'.format(replica_dir, sample_id, new_time)
    export_df.to_csv(export_path, sep = ' ')

    c = sample_cols[sample_no]
    r = sample_rows[sample_no]
    fig.add_trace(go.Scatter(
      x=df.Energy_keV, 
      y=replica_spectrum, 
      mode = 'lines',
      line = go.scatter.Line(color = 'blue', width = 0.1),
      name = None),
      row=r, col=c
      )
    sample_no += 1

print('Hotovo.')

fig.update_layout(height=1200, width=1000,
                  showlegend = False,
                  title_text=r'Zdrojová spektra a repliky')
fig.show()

Tyto grafy slouží pro kontrolu správné generace dat. Protože jednotlivé repliky se mírně liší časem měření, mají replikovaná spektra různé škály. Mají ale stejné aktivity a tedy spočtené výtěžky z rentgenových píků by měly být pro všechny repliky stejné. 

## 3. Download

Data jsou vygenerovaná, ale nachází se na cloudu Google Colabu, odkud je chceme stáhnout. Tento krok nejspolehlivěji provedeme manuálně. 
1. Zazipujeme vygenerované soubory 

In [ ]:
#@title <-- Zazipujte vygenerovaná data

!zip -r Replicas.zip ./Replicas


2. Klikněte na `View -> Table of Contents`. 
3. Otevře se panel vlevo (mimochodem vůbec není šptatný nápad mít ho otevřený stále). 
4. Kliknout na 📁 úplně nalevo, a zobrazí se seznam souborů. 
5. Najedeme myší na soubor `Replicas.zip` a klikneme na symbol **⁝** vpravo, zvolíme `Upload` a stahneme ho do svého počítače. 

## Operace se soubory

Většinu lokálních operací s vygenerovanými soubory je nejpohodlnější udělat přímo z notebooku. Stačí si otebřít buňku pro kód (`Ctrl-M B) a zadat příslušný příkaz s vykřičníkem.

Níže uvádím příklady. 

**Důležité**  U této buňky **neklikejte** na šipku pro vykonání, spouštějte jenom potřebné řádky (označit řádek + `Ctrl-Shift-Enter`) nebo obsah vymažte a zadejte příkaz, který potřebujete. 

In [ ]:
# Obsah lokálního adresáře notebooku
!ls
# Smazat všechna vygenerovaná data
!rm -rf ./Replicas
# Smazat všechny načtené textové soubory
!rm -f *.txt